In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop , Adam
from tensorflow.keras import regularizers

In [ ]:
file_path="/content/sample_data/Student_performance_data _.csv"
column_type=[]
for i in range(15):
  if i==0:
    column_type.append(tf.string)
  else:
    column_type.append(tf.float32)

In [ ]:
dataset= tf.data.experimental.CsvDataset(file_path, record_defaults=column_type, header=True)
def preprocessing(dataset):
  # Convert the dataset to a list of lists to make it subscriptable
  dataset_list = list(dataset.as_numpy_iterator())
  features=[]
  labels=[]
  for row in dataset_list:
    features.append(row[1:13])
    labels.append(row[-1])
  # Normalize features (assuming you want to normalize each feature column-wise)
  features_np = np.array(features)  # Convert to NumPy array for easier manipulation
  print(features_np.shape)
  for i in range(features_np.shape[1]):  # Iterate over columns
    features_np[:, i] = (features_np[:, i] - features_np[:, i].min()) / (features_np[:, i].max() - features_np[:, i].min())

  return features_np, np.array(labels)
x,y = preprocessing(dataset)
x=np.array(x)
y=np.array(y)



NameError: name 'tf' is not defined

In [ ]:
model = Sequential()
model.add(Dense(256,activation='leaky_relu',input_shape=(12,),kernel_regularizer=regularizers.l2(0.03)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(64,activation='elu'))
model.add(BatchNormalization())
model.add(Dense(32,activation='elu'))
model.add(BatchNormalization())
model.add(Dense(16,activation="leaky_relu"))
model.add(BatchNormalization())
model.add(Dense(5,activation='softmax'))
model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_31 (Dense)                     │ (None, 256)                 │           3,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_27               │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,941 (93.52 KB)

 Trainable params: 23,205 (90.64 KB)

 Non-trainable params: 736 (2.88 KB)

In [ ]:
train_size = int(0.85 * len(x))
test_size = len(x) - train_size
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
Early_stopping=EarlyStopping(monitor='val_loss', patience=4)
history = model.fit(x_train, y_train, epochs=100, batch_size=100, validation_split=0.15,callbacks=[reduce_lr,Early_stopping])

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.8128 - loss: 0.3960 - val_accuracy: 0.7934 - val_loss: 0.4382 - learning_rate: 1.0000e-04
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8472 - loss: 0.3649 - val_accuracy: 0.7836 - val_loss: 0.4368 - learning_rate: 1.0000e-04
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8534 - loss: 0.3620 - val_accuracy: 0.7934 - val_loss: 0.4286 - learning_rate: 1.0000e-04
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8671 - loss: 0.3469 - val_accuracy: 0.8000 - val_loss: 0.4262 - learning_rate: 1.0000e-04
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8451 - loss: 0.3715 - val_accuracy: 0.8066 - val_loss: 0.4270 - learning_rate: 1.0000e-04
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8419 - loss: 0.3659 - val_accuracy: 0.8033 - val_loss: 0.4276 - learning_rate: 1.0000e-04
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8584

In [ ]:
a = 3703
while True:
  print(a%5,   a//5)
  a = a//5
  if a in range(5):
    print(a)
    break

3 740
0 148
3 29
4 5
0 1
1


In [ ]:
a = 0.106
print(0.176 * 16)

2.816


In [ ]:
a=0
a += 5*(9**3)
a+= 6*(9**1)
a+= 4
print(a)

3703
